In [2]:
import pandas as pd
import pickle
pd.set_option('display.max_colwidth', None)

from spacy.language import Language


import es_core_news_sm
nlp = es_core_news_sm.load()
from spacy.lang.es.stop_words import STOP_WORDS

import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from sklearn.decomposition import NMF

%pylab inline

Populating the interactive namespace from numpy and matplotlib


#### Load data set

In [3]:
data = pickle.load( open( "../data/Voc_strat_text.pickle", "rb" ) )

In [4]:
data.head()

,Index_survey,A-Q1,n_words,clean_text,tokens,lemmas
0,0,Similitud con otros idiomas. Composición con alguna otra palabra en alemán que conozco. Contexto.,14,similitud con otros idiomas. composicion con alguna otra palabra en aleman que conozco. contexto.,"[similitud, otros, idiomas, composicion, otra, aleman, conozco, contexto]","[similitud, idioma, composicion, aleman, conocer, contexto]"
1,1,"Relaciono terminaciónes, palabras compuestas de las que conozco al menos el significado de una de las partes. Por el contexto: habla de un campamento, por lo que usará vocabulario relacionado.",30,"relaciono terminaciones, palabras compuestas de las que conozco al menos el significado de una de las partes. por el contexto: habla de un campamento, por lo que usara vocabulario relacionado.","[relaciono, terminaciones, compuestas, conozco, partes, contexto, campamento, usara, vocabulario, relacionado]","[relacionar, terminación, componer, conocer, partir, contexto, campamento, usar, vocabulario, relacionar]"
2,2,"Estoy en un A1.1 de alemán o sea que no sé demasiado pero lo que hago es leer la frase entera y tratar de buscar el contexto en el que puede estar. En el caso de no lograrlo, lo poco que llevo visto de alemán me hace pensar que muchas cosas las puedo sacar del Inglés como, por ejemplo, bed y Bett, o sea que las pronuncio en voz alta como yo creo que se pronuncian para buscar esa similitud con el Inglés. También ayuda mucho el hecho de que los sustantivos siempre van en mayúsculas, por lo que ahí yo ya encuentro una pequeña pista para saber qué puede ser.",111,"estoy en un a1.1 de aleman o sea que no se demasiado pero lo que hago es leer la frase entera y tratar de buscar el contexto en el que puede estar. en el caso de no lograrlo, lo poco que llevo visto de aleman me hace pensar que muchas cosas las puedo sacar del ingles como, por ejemplo, bed y bett, o sea que las pronuncio en voz alta como yo creo que se pronuncian para buscar esa similitud con el ingles. tambien ayuda mucho el hecho de que los sustantivos siempre van en mayusculas, por lo que ahi yo ya encuentro una pequeña pista para saber que puede ser.","[a1, aleman, leer, frase, entera, contexto, caso, lograrlo, llevo, visto, aleman, pensar, sacar, ingles, bed, bett, pronuncio, voz, alta, pronuncian, similitud, ingles, sustantivos, mayusculas, encuentro, pequeña, pista]","[aleman, leer, frase, entero, contexto, casar, lograrlo, llevar, vestir, aleman, pensar, sacar, ingle, bed, bett, pronunciar, voz, alto, pronunciar, similitud, ingle, sustantivo, mayusculas, encontrar, pequeño, pistar]"
3,3,"Principalmente me fijo en la terminación de la palabra, ya que sé que cuando acaba en ung, eit, ion y demás sufijos suelen ser nombres abstractos, aunque para descubrir la categoría de palabra me fijo principalmente en el contexto, y en caso de que sea palabra compuesta me fijo en cualquiera de las que conozca y trato de intuir qué significa. En este caso ""abgeschirmt"" es desconocida para mí, pero por el contexto y el tema que estamos tratando podría significar inhibir u obviar. Me ayuda principalmente encontrar palabras difíciles en oraciones con una sintaxis simples y con palabras conocidas. en este caso el texto tiene demasiadas palabras desconocidas en algunas partes para poder deducir el significado por contexto, así que imagino lo que significa o principalmente lo omito.",129,"principalmente me fijo en la terminacion de la palabra, ya que se que cuando acaba en ung, eit, ion y demas sufijos suelen ser nombres abstractos, aunque para descubrir la categoria de palabra me fijo principalmente en el contexto, y en caso de que sea palabra compuesta me fijo en cualquiera de las que conozca y trato de intuir que significa. en este caso ""abgeschirmt"" es desconocida para mi, pero por el contexto y el tema que estamos tratando podria significar inhibir u obviar. me ayuda principalmente encontrar palabras dificiles en oraciones con una sintaxis simples y con palabras conocidas. en este

#### Stop-words

In [5]:
not_stop_words = ['breve','cuanta', 'cuantas', 'cuanto', 'cuantos', 'diferente', 'diferentes', 'donde',
                 'enseguida', 'excepto', 'junto', 'otra', 'otras', 'otro', 'otros', 'pasada', 'pasado', 'propia', 'propias',
                 'propio', 'propios', 'proximo','proximos', 'siguiente', 'ultimo', 'ultima', 'ultimos', 'ultimas']
# general, gran, grandes, incluso, lado, largo, mediante, mientras, mucho/a/s, ningun/o/a/s, posible, primer/a/s, principalmente,
# raras, segunda/o, tambien, tercera, toda/o/s
# como, ejemplo

for word in not_stop_words:
    nlp.vocab[word].is_stop = False

STOP_WORDS = [word for word in STOP_WORDS if word not in not_stop_words]

In [6]:
new_stop_words = ['palabra', 'palabras', 'significa', 'significan', 'significar', 'significado', 'significados', 'averiguo', 'averiguar',
                 'busco','buscar','descubro', 'descubrir', 'desconocida', 'desconocidas', 'desconocido', 'desconocidos',
                'desconozco','trato','tratar','intento','intentar','suelo','normalmente','ayuda','ayudan', 'gracias',
                 'entender','entenderla','entenderlas','entenderlo','comprender','comprenderla','comprenderlas',
                 'uso','usar','utilizo','utilizar','utilizando','algun','estrategia','estrategias','tecnica','tecnicas',
                 'miro','mirar','etc']

for word in new_stop_words:
    nlp.vocab[word].is_stop = True

STOP_WORDS += new_stop_words

#### Functions

In [7]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [8]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [9]:
def NMF_results(line, Z_transf):
    res = pd.DataFrame(columns=['txt','EL_CON','SIM_ID','DICC','CON_AM'])
    
    txt = line['A-Q1'].values
    ind = line.index[0]
    vec = Z_transf[ind,:]*100 / Z_transf[ind,:].sum()

    return list(vec)+list(txt)

## Td-Idf

In [10]:
datacl = data['clean_text']

vectorizer = TfidfVectorizer(ngram_range=(1,1),stop_words=STOP_WORDS, min_df=5)
docs = vectorizer.fit_transform([str(l) for l in list(datacl.values)])
docs_arr = docs.toarray()
features   = vectorizer.get_feature_names()
Z=pd.DataFrame(docs_arr, columns=features)

In [11]:
Z.head()

,acompaña,adivinar,adjetivo,adjetivos,aleman,alemanas,analizar,analizo,aproximado,articulo,...,ung,util,veo,verbo,verbos,viene,visto,vocabulario,whatsapp,yoga
0,0.0,0.0,0.0,0.0,0.345450,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.389066,0.0,0.0
2,0.0,0.0,0.0,0.0,0.378828,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.316889,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.21187,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


## NMF

In [12]:
n_topics = 4
nmfFR = NMF(n_components=n_topics, random_state=0,
          alpha=.1, l1_ratio=.5).fit(Z)

In [13]:
voc_top = pd.DataFrame(nmfFR.components_, columns=features,index=['EL_CON','SIM_ID','DICC','CON_AM']).T

In [14]:
print_top_words(nmfFR,features , 15)

Topic #0: contexto conozco compuesta otras sacar fijo verbo compuestas frase conozca oracion deducir entiendo raiz otra
Topic #1: ingles idiomas español conozco otros otras parecido aleman similitud lenguas similares parecidas parecen frances deducir
Topic #2: diccionario idioma otro traductor aleman online comprendo resulta similitud intuir otras fijo completa pons leo
Topic #3: texto frase leo entiendo leer idea tema verbos sentido deducir caso compuestas resto comprension tipo



### Data Transformation

In [15]:
Z_transf = nmfFR.transform(Z)

NMF_data = pd.DataFrame(columns=['Index_survey','Topic_0','Topic_1','Topic_2','Topic_3'])

In [16]:
NMF_data['Index_survey'] = data.Index_survey.values
NMF_data[['Topic_0','Topic_1','Topic_2','Topic_3']] = Z_transf
NMF_data_n = NMF_data.rename(columns={'Topic_0':'EL_CON',
                                     'Topic_1':'SIM_ID',
                                     'Topic_2':'DICC',
                                     'Topic_3':'CON_AM'})

In [17]:
#NMF_data_n.to_csv('data/NMF_topic_n.csv', index=False)

### Examples

In [18]:
inds = [4, 75, 78, 93, 354, 369, 142]

In [19]:
examples = pd.DataFrame(columns=['EL_CON', 'SIM_ID', 'DICC', 'CON_AM','txt'])

In [20]:
for el in inds:
    examples.loc[el] = NMF_results(data[data.Index_survey == el], Z_transf)

In [21]:
examples

,EL_CON,SIM_ID,DICC,CON_AM,txt
4,0.000000,100.000000,0.000000,0.000000,Relacionarlas con palabras conocidas de mi lengua materna o de las lenguas que conozco y aprendo. La ortografía de las palabras o su sonido puede recordarme a otras palabras similares en otros idiomas.
75,34.145618,0.000000,0.000000,65.854382,"Me fijo en el contexto, trato de averiguar el significado a partir de la etimología o de las palabras que forman los compuestos. Trato de entender el texto globalmente para comprender el tema del que se habla antes de imaginar lo que puede significar cada término."
78,13.453096,60.981811,0.000000,25.565093,"Primero busco palabras que se parezcan a los idiomas que ya domino, como el español o el inglés y después intento entender qué significan las otras palabras gracias al contexto o a información que se me de acerca del texto."
93,26.422278,0.000000,53.627408,19.950314,"Por lo general, si lo tengo a mano, tiendo a buscar palabras desconocidas en el diccionario, aunque no todas, ya que muchas de ellas consigo entenderlas por el contexto de la situación mientras estoy leyendo. Cuando no tengo el diccionario a mano, me fijo, lo primero, en si es un nombre o no, y dependiendo de eso, intento darle vueltas al significado teniendo en cuenta el contexto y mirando si es una palabra simple o compuesta, la función que está desempeñando en la frase, etc. A partir de ahí, si no obtengo el significado concreto, siempre consigo, al menos, una idea de lo que quiere decir."
354,97.959573,0.000000,0.000000,2.040427,"A mi en lo personal, me ayuda el contexto en caso de no conocer cierta palabra. Y si ya ni con la ayuda del contexto lo puedo averiguar, intento separar la palabra si es compuesta o buscar su procedencia."
369,0.000000,0.000000,100.000000,0.000000,Aprendo palabras desconocidas con el diccionario y subrayandolas posteriormente.
142,58.255317,0.000000,0.000000,41.744683,Contexto de la frase
